In [1]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.preprocessing import scale
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler

from scipy.stats import spearmanr
from scipy.stats import pearsonr
import scipy.cluster.hierarchy as sch
import talib

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import coin_data as cd
import coin_plots as cp
import market_alpha as ma
import formulaic_alpha as fa

from pyfinance.ols import PandasRollingOLS

sns.set_style('whitegrid')
idx = pd.IndexSlice

from pytrends.request import TrendReq

In [2]:
btcusd = cd.process('cryptocompare', 'BTCUSD')
btcusd = btcusd['2015':].copy()

In [3]:
btcusd.head()

,high,low,open,volumefrom,volumeto,close,unrealized_profit,unrealized_loss,mvrv,mvrv_z_score,...,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01,321.36,313.54,318.24,12820.26,4073067.10,314.89,0.364847,0.381237,0.984037,-0.023789,...,152648.0,102135.882070,3.214997e+07,2.786937,877.262055,0.019577,6.162407,16.104075,0.029546,0.024775
2015-01-02,316.40,313.08,314.89,14728.97,4673971.80,315.21,0.365072,0.376849,0.989531,-0.015602,...,241133.0,149397.417177,4.720103e+07,2.867718,906.034662,0.012090,3.819748,16.099986,0.029541,0.024736
2015-01-03,315.83,284.89,315.21,47366.46,14209564.70,287.13,0.357438,0.492721,0.888807,-0.139012,...,266646.0,277500.536361,7.926728e+07,3.084672,881.128133,0.004608,1.316124,16.094621,0.030392,0.030157
2015-01-04,289.94,255.87,287.13,88769.22,24255392.10,264.72,0.353442,0.552672,0.830200,-0.252265,...,244195.0,226786.020740,5.972060e+07,3.787183,997.296192,0.010009,2.635763,16.112310,0.030924,0.032231
2015-01-05,278.32,262.89,264.72,54993.28,14995382.75,274.84,0.356353,0.501804,0.873019,-0.188308,...,264832.0,254454.056878,6.997585e+07,3.914129,1076.400546,0.021808,5.997284,16.119254,0.031131,0.033784


In [14]:
del data

In [15]:
data = btcusd[['high', 'low', 'open', 'close', 'volumeto']]

In [12]:
ma.market(data, 20, 50, 200)

,high,low,open,close,SMA_20,SMA_50,SMA_200,mayer_mult,EMA_20,EMA_50,...,CMO,RSI14,RSI90,fastk,fastd,STOCH,ULTOSC,WILLR,ATR,NATR
time,,,,,,,,,,,,,,,,,,,,,
2015-02-19,244.11,234.87,235.20,241.92,229.6170,241.3638,NaN,NaN,234.856332,241.363800,...,2.966645,51.483322,NaN,60.328755,58.810866,0.939450,54.574446,-45.182525,17.458904,7.216809
2015-02-20,247.76,238.83,241.92,244.52,230.9175,239.9564,NaN,NaN,235.776682,241.487573,...,4.988024,52.494012,NaN,66.164245,58.792070,0.963415,52.987640,-39.996010,16.849696,6.890928
2015-02-21,247.32,242.65,244.52,244.41,231.8735,238.5404,NaN,NaN,236.598902,241.602178,...,4.888470,52.444235,NaN,65.876845,64.123282,0.925373,46.744584,-40.215440,15.979718,6.538079
2015-02-22,246.54,232.72,244.41,235.73,231.7170,237.5124,NaN,NaN,236.516150,241.371896,...,-2.933230,48.533385,NaN,43.296498,58.445863,1.013206,51.035515,-57.530421,15.825452,6.713381
2015-02-23,240.06,231.33,235.73,238.82,232.2875,236.9944,NaN,NaN,236.735564,241.271822,...,-0.072278,49.963861,NaN,51.555738,53.576360,1.077516,52.956909,-51.366447,15.318634,6.414301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-02,19394.33,18929.91,19313.96,19058.15,19394.2310,20408.4382,28037.33095,0.679742,19504.726243,20355.749412,...,-13.317488,43.341256,43.742972,60.257592,75.798101,1.120749,50.035705,-60.310030,882.030312,4.628100
2022-10-03,19697.23,18988.80,19058.15,19632.35,19367.1600,20314.8110,27930.70875,0.702895,19516.880887,20327.380808,...,-1.178324,49.410838,44.316134,100.000000,79.885240,1.011827,50.202314,-34.061247,869.630289,4.429578
2022-10-04,20459.26,19499.64,19632.35,20344.58,19372.7050,20239.6688,27823.47680,0.731202,19595.709374,20328.055286,...,11.487622,55.743811,45.018759,100.000000,86.752531,0.839503,61.578964,-5.053452,876.058126,4.306101


In [16]:
fa.formulaic(data)

,high,low,open,close,volumeto,002,003,004,005,006,...,075,078,081,083,085,086,094,095,099,101
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01,321.36,313.54,318.24,314.89,4.073067e+06,NaN,NaN,NaN,NaN,NaN,...,0,NaN,0,NaN,NaN,0,NaN,0,NaN,-0.428334
2015-01-02,316.40,313.08,314.89,315.21,4.673972e+06,NaN,NaN,NaN,NaN,NaN,...,0,NaN,0,NaN,NaN,0,NaN,0,NaN,0.096357
2015-01-03,315.83,284.89,315.21,287.13,1.420956e+07,NaN,NaN,NaN,NaN,NaN,...,0,NaN,0,NaN,NaN,0,NaN,0,NaN,-0.907534
2015-01-04,289.94,255.87,287.13,264.72,2.425539e+07,NaN,NaN,NaN,NaN,NaN,...,0,NaN,0,NaN,NaN,0,NaN,0,NaN,-0.657744
2015-01-05,278.32,262.89,264.72,274.84,1.499538e+07,NaN,NaN,NaN,NaN,NaN,...,0,NaN,0,NaN,NaN,0,NaN,0,NaN,0.655823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-02,19394.33,18929.91,19313.96,19058.15,4.857993e+08,-0.113015,0.226540,-7.0,10404.028359,0.226540,...,0,NaN,-1,8.842281e+06,0.901516,0,-2.843475e+02,0,0.966803,-0.550815
2022-10-03,19697.23,18988.80,19058.15,19632.35,1.106456e+09,-0.836251,0.233195,-7.0,-52466.393156,0.233195,...,0,NaN,-1,-1.594298e+06,0.802679,-1,-4.543925e+02,0,0.987311,0.810524
2022-10-04,20459.26,19499.64,19632.35,20344.58,1.290593e+09,-0.881397,0.164659,-9.0,108979.308099,0.164659,...,0,NaN,-1,-1.749376e+06,0.850965,-1,-1.094218e+03,0,0.991998,0.742199
